In [2]:
import os
os.environ['kdconfig']='database.yaml'
import pandas as pd
from qsdata.api import get_trading_dates,index_components,get_price,get_vwap
from datetime import date
from qsdata import kddb
import datetime
from tqdm import tqdm

### 新闻数据

In [3]:
sql = "select * from kd_news_for_em where date_time >= '2021-01-01' and date_time < '2021-02-01'"
news = kddb.read_db(sql,db_from="to_pull_news")
news

,id,event_type,obj_id,source_obj_id,source,title,short_title,url,date_time,create_time,...,top_kd,top_em,tag,img_type,img_url,stock_yn,industry_yn,macro_yn,news_type,remark
0,28121269,news,99113744447,0.0,,长春高新出现30笔大宗交易 折价率均为7.55%,None,em-139115743,2021-01-01 00:12:00,2021-01-01 14:29:18,...,0,0,,,,1,0,0,3,None
1,28121295,news,99113744476,0.0,,纳川股份出现1笔大宗交易 折价率为14.56%,None,em-139115781,2021-01-01 00:12:00,2021-01-01 14:29:18,...,0,0,,,,1,0,0,3,None
2,28121317,news,99113744500,0.0,,淳中科技出现2笔大宗交易 折价率均为13.99%,None,em-139115830,2021-01-01 00:12:00,2021-01-01 14:29:18,...,0,0,,,,1,0,0,3,None
3,28121375,news,99113744584,0.0,,重磅发布 | 观点指数·2020中国房地产销售TOP100,None,https://finance.sina.com.cn/stock/stockzmt/202...,2021-01-01 00:12:00,2021-01-01 14:29:19,...,0,0,,,,0,1,1,1,None
4,28121537,news,99113744806,0.0,,土地热线 | 华发74.69亿再入江城 融创65亿包揽绍兴黄酒小镇,None,https://finance.sina.com.cn/stock/stockzmt/202...,2021-01-01 00:12:00,2021-01-01 14:29:20,...,0,0,,,,0,1,0,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206445,28328909,news,99114067263,0.0,,【纺织头条】聚酯厂家年底大促销、恒力加油站开业、纺织集群“就地过年”奖励出台……放假归放假，...,None,http://mp.weixin.qq.com/s?__biz=MjM5NzAzMzU4MA...,2021-01-30 23:11:58,2021-01-31 00:35:07,...,0,0,,exCover,https://newsimg.emoney.cn/7fd483a94a65beb7dc47...,0,1,0,2,None
206446,28328910,news,99114067264,0.0,,科普|各式纤维织物的性能特点知多少？,None,http://mp.weixin.qq.com/s?__biz=MjM5NzAzMzU4MA...,2021-01-30 23:11:58,2021-01-31 00:35:07,...,0,0,,exSide,https://newsimg.emoney.cn/2b04ca8ba1430563d9cd...,0,1,0,2,None
206447,28328784,news,99114067105,0.0,,【光大固收】低估值优质标的仍是投资的主要方向——可转债周报（2021年1月25日至1月29日）,None,http://mp.weixin.qq.com/s?__biz=MzA4NjIyMTE1Ng...,2021-01-30 23:11:59,2021-01-30 23:35:08,...,0,0,,exCover,https://newsimg.emoney.cn/b149df9e1f8636943cf2...,0,0,1,1,None
206448,28328911,news,99114067265,0.0,,关注|印度：纺织业原材料供应紧张，或影响数十万工人生计,None,http://mp.weixin.qq.com/s?__biz=MjM5NzAzMzU4MA...,2021-01-30 23:11:59,2021-01-31 00:35:07,...,0,0,,exSide,https://newsimg.emoney.cn/4d564094df56490637c5...,0,1,1,1,None


In [4]:
news.to_pickle("1mnews.pkl")

### 量价数据 

In [10]:
# 方法一：sql
sql = "select * from kdb_dayquota where dt >= '2018-01-01'"
daily_df = kddb.read_db(sql,db_from="kdbase")
daily_df

,kdcode,dt,prev_close,close,open,high,low,volume,turnover,dealnum,...,vwap,adjfactor,upprice,downprice,trade_status,free_float_shares,total_shares,update_dt,update_time,realtime_open
0,000001.SH,2018-01-02,3307.1721,3348.3259,3314.0307,3349.0530,3314.0307,2.022789e+10,2.277885e+11,NaN,...,11.261111,1.0,NaN,NaN,0,NaN,NaN,2019-05-24 09:12:07,2020-10-23 15:58:49,3314.0307
1,000001.SH,2018-01-03,3348.3259,3369.1084,3347.7428,3379.9152,3345.2887,2.138361e+10,2.583665e+11,NaN,...,12.082453,1.0,NaN,NaN,0,NaN,NaN,2019-05-24 09:12:02,2020-10-23 15:58:49,3347.7428
2,000001.SH,2018-01-04,3369.1084,3385.7102,3371.0000,3392.8264,3365.2954,2.069553e+10,2.430908e+11,NaN,...,11.746053,1.0,NaN,NaN,0,NaN,NaN,2019-05-24 09:12:03,2020-10-23 15:58:49,3371.0000
3,000001.SH,2018-01-05,3385.7102,3391.7501,3386.4640,3402.0694,3380.2450,2.130607e+10,2.481878e+11,NaN,...,11.648693,1.0,NaN,NaN,0,NaN,NaN,2019-05-24 09:12:07,2020-10-23 15:58:49,3386.4640
4,000001.SH,2018-01-08,3391.7501,3409.4795,3391.5528,3412.7307,3384.5591,2.361651e+10,2.862132e+11,NaN,...,12.119200,1.0,NaN,NaN,0,NaN,NaN,2019-05-24 09:12:00,2020-10-23 15:58:49,3391.5528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116677,SPX.GI,2020-02-04,3248.9200,3297.5900,3280.6100,3306.9200,3280.6100,3.995320e+09,NaN,NaN,...,NaN,1.0,NaN,NaN,0,NaN,NaN,2020-02-06 22:00:03,2020-10-23 15:58:49,3280.6100
3116678,SPX.GI,2020-02-05,3297.5900,3334.6900,3324.9100,3337.5800,3313.7500,4.117730e+09,NaN,NaN,...,NaN,1.0,NaN,NaN,0,NaN,NaN,2020-02-07 22:00:02,2020-10-23 15:58:49,3324.9100
3116679,SPX.GI,2020-02-06,3334.6900,3345.7800,3344.9200,3347.9600,3334.3900,3.868370e+09,NaN,NaN,...,NaN,1.0,NaN,NaN,0,NaN,NaN,2020-02-07 22:00:02,2020-10-23 15:58:49,3344.9200
3116680,SPX.GI,2020-02-07,3345.7800,3327.7100,3335.5400,3341.4200,3322.1200,3.730650e+09,NaN,NaN,...,NaN,1.0,NaN,NaN,0,NaN,NaN,2020-02-08 22:00:02,2020-10-23 15:58:49,3335.5400


In [3]:
# 方法二：qsdata
# 获取日线数据

s_dt = date(2020,1,1)
e_dt = date(2021,10,31)
features = ["open", "close", "high", "low", "turnover", "volume"]

daily_data = get_price(None,
              start_date=s_dt,
              end_date=e_dt,
              fields=features,
              frequency='1d',
              adjust_type = "post")

daily_data = daily_data.reset_index()

daily_data['dt'] = daily_data['dt'].apply(lambda x: x.date())
daily_data.to_pickle('22mdata.pkl')
daily_data

,kdcode,dt,open,close,high,low,turnover,volume
0,000001.SH,2020-01-02,3066.3357,3085.1976,3098.1001,3066.3357,3.271971e+11,2.924702e+10
1,000001.SH,2020-01-03,3089.0220,3083.7858,3093.8192,3074.5178,2.899917e+11,2.614967e+10
2,000001.SH,2020-01-06,3070.9088,3083.4083,3107.2032,3065.3088,3.311825e+11,3.125758e+10
3,000001.SH,2020-01-07,3085.4882,3104.8015,3105.4507,3084.3290,2.881592e+11,2.765831e+10
4,000001.SH,2020-01-08,3094.2389,3066.8925,3094.2389,3059.1313,3.065174e+11,2.978726e+10
...,...,...,...,...,...,...,...,...
1974404,SPX.GI,2020-02-04,3280.6100,3297.5900,3306.9200,3280.6100,NaN,3.995320e+09
1974405,SPX.GI,2020-02-05,3324.9100,3334.6900,3337.5800,3313.7500,NaN,4.117730e+09
1974406,SPX.GI,2020-02-06,3344.9200,3345.7800,3347.9600,3334.3900,NaN,3.868370e+09
1974407,SPX.GI,2020-02-07,3335.5400,3327.7100,3341.4200,3322.1200,NaN,3.730650e+09
